### Test out the new logger function

First test was to make sure that the logger only initiates (or at the very least makes a message) only once. Had to make the logger metaclass=Singleton. Totally don't understand that. 

In [1]:
from fireatlas import postprocess, settings, FireLog, FireConsts
from fireatlas.FireLog import logger
import os

2024-08-14 17:48:41,298 - fireatlas.FireLog - INFO - logger initialized!


In [2]:
# also need to set my env vars, will need to restart to take effect
import dotenv 

envpath = os.path.join(FireConsts.root_dir, '.env')
dotenv.set_key(envpath, 'FEDS_READ_LOCATION', 'local')
print(settings.READ_LOCATION)


local


Now let's test out it's ability to change the path of the file handler. Make its subdirectories defined by year and region. 

In [3]:
# in a typical script, there should be region and year already defined as global vars
# define them here
region = ['Zogg', 'Zogg_perimeter']
tst = [2020, 9, 15, 'AM']
location = None


Show it's file handler path intially, change it to a subdirectory, reset it.

In [4]:
print(logger.handlers)
newpath = postprocess.all_dir(tst, region, location)
FireLog.update_fh(logger, newpath) # logger.update_fh_dir(newpath)
print(logger.handlers)
FireLog.reset_fh(logger) # logger.reset_fh()
print(logger.handlers)


[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\running.log (INFO)>]
[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\data\FEDSoutput-v3\Zogg\2020\running.log (INFO)>]
[<StreamHandler stderr (INFO)>, <FileHandler D:\fireatlas\running.log (INFO)>]


Next, need to show that the logger will change file handler path when the region and year changes. I suspect this will have no issues when done in serial, but question remains when done in parallel with `dask`. 

In [5]:
# create a list of "fires" to process
region_list = [['castle', 'castle_perim'], 
               ['caldor', 'caldor_perim'], 
               ['mosquito', 'mosquito_perim'], 
               ['zogg', 'zogg_perim']]
tst_list = [
    [2020, 8], [2021, 7], [2022, 8], [2020, 9]
]
location = None

In [6]:
# create a function to apply to each iteration
# does the logger properly log in the right subdirectory
# can it also log to the main directory when it's not region/year specific? 
def test_logger(logger, tst, region, location):
    # log into the main file handler
    FireLog.reset_fh(logger) # logger.reset_fh()
    logger.info("Processing something for all fires.")

    # log into a subdirectory
    subdir = postprocess.all_dir(tst, region, location)
    FireLog.update_fh(logger, subdir) # logger.update_fh_dir(subdir)
    os.makedirs(subdir, exist_ok=True)
    logger.info(f"logging into the {region[0]} fire.")
    

Logger works great when used in serial. Check out the running.log files in the data directory.

In [7]:
for tst, region in zip(tst_list, region_list):
    test_logger(logger, tst, region, location)

2024-08-14 17:48:42,124 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-14 17:48:42,165 - fireatlas.FireLog - INFO - logging into the castle fire.
2024-08-14 17:48:42,171 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-14 17:48:42,207 - fireatlas.FireLog - INFO - logging into the caldor fire.
2024-08-14 17:48:42,230 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-14 17:48:42,262 - fireatlas.FireLog - INFO - logging into the mosquito fire.
2024-08-14 17:48:42,268 - fireatlas.FireLog - INFO - Processing something for all fires.
2024-08-14 17:48:42,273 - fireatlas.FireLog - INFO - logging into the zogg fire.


Test out the logger when using `dask`!

In [8]:
from dask.distributed import Client
from functools import partial
import logging

# EDIT: only had to do this when logger was a class. 
# # no idea how to avoid doing this
# all_loggers = logging.root.manager.loggerDict
# for logger_name, logger in all_loggers.items():
#     if logger_name.startswith('fireatlas') and isinstance(logger, logging.Logger):
#         logger.setLevel(logging.ERROR)

# start dask client
client = Client(n_workers = 4, threads_per_worker = 1)

print(client.dashboard_link)

http://127.0.0.1:8787/status


In [9]:
# try it out 
futures = client.map(partial(test_logger, logger, location=location), tst_list, region_list)
results = client.gather(futures)

In [10]:
client.close()

### Test on FEDS

It's one thing to test on isolated logging. Now let's test it with running FEDS. 

First, without changing the logging path, works fine. 

In [11]:
from fireatlas import FireRun, settings
import os
from fireatlas.FireLog import logger

# make sure logs go to the root directory (should be false)
print(settings.LOG_SUBDIR)

False


In [12]:
# now run 
perim_path = os.path.join(settings.dirextdata, 'perimeters', 'zogg.fgb')
try:
    FireRun.constrainByShape_Run(perim_path)
except Exception as e:
    logger.exception(e)


2024-08-14 17:49:15,676 - fireatlas.FireLog - INFO - Preprocessing has already occurred for this landcover file.
2024-08-14 17:49:15,677 - fireatlas.FireLog - INFO - func:preprocess_landcover took: 1.04 ms
2024-08-14 17:49:16,020 - fireatlas.FireLog - INFO - =============== Running: ZOGG_00009978 ===============
2024-08-14 17:49:16,425 - fireatlas.FireLog - INFO - func:preprocess_region took: 402.94 ms
2024-08-14 17:49:16,450 - fireatlas.FireLog - INFO - func:read_region took: 21.13 ms
2024-08-14 17:49:16,465 - fireatlas.FireLog - INFO - func:read_region took: 12.11 ms
2024-08-14 17:49:16,466 - fireatlas.FireLog - INFO - filtering and clustering 2020-9-27 AM, SNPP, ZOGG_00009978
2024-08-14 17:49:16,519 - fireatlas.FireLog - INFO - func:read_preprocessed_input took: 49.95 ms
2024-08-14 17:49:16,555 - fireatlas.FireLog - INFO - func:preprocess_region_t took: 103.22 ms
2024-08-14 17:49:16,568 - fireatlas.FireLog - INFO - func:read_region took: 11.11 ms
2024-08-14 17:49:16,569 - fireatlas.

In [14]:
# set the constant for logging in subdirectories to True
import dotenv 
from fireatlas import FireConsts
from fireatlas import FireRun, settings
import os
from fireatlas.FireLog import logger

envpath = os.path.join(FireConsts.root_dir, '.env')
dotenv.set_key(envpath, 'FEDS_LOG_SUBDIR', 'true')
print(settings.LOG_SUBDIR)


False


In [19]:
import shutil

# also delete logs and data
os.remove(os.path.join(FireConsts.root_dir, 'running.log'))
shutil.rmtree(os.path.join(settings.diroutdata, 'ZOGG_00009978'))
shutil.rmtree(os.path.join(settings.get_path(None), settings.PREPROCESSED_DIR, 'ZOGG_00009978'))


In [6]:
# redo this
perim_path = os.path.join(settings.dirextdata, 'perimeters', 'zogg.fgb')
try:
    FireRun.constrainByShape_Run(perim_path)
except Exception as e:
    logger.exception(e)

2024-08-14 16:43:57,612 - fireatlas.FireLog - INFO - Preprocessing has already occurred for this landcover file.
2024-08-14 16:43:57,619 - fireatlas.FireLog - INFO - func:preprocess_landcover took: 6.53 ms
2024-08-14 16:43:58,668 - fireatlas.FireLog - INFO - =============== Running: ZOGG_00009978 ===============
2024-08-14 16:43:58,671 - fireatlas.FireLog - INFO - Preprocessing has already occurred for this region.
2024-08-14 16:43:58,673 - fireatlas.FireLog - INFO - func:preprocess_region took: 3.00 ms
2024-08-14 16:43:58,703 - fireatlas.FireLog - INFO - func:read_region took: 22.95 ms
2024-08-14 16:43:58,705 - fireatlas.FireLog - ERROR - 'Logger' object has no attribute 'fh'
Traceback (most recent call last):
  File "C:\Users\lrosenth\AppData\Local\Temp\ipykernel_19056\1975649432.py", line 4, in <module>
    FireRun.constrainByShape_Run(perim_path)
  File "D:\fireatlas\fireatlas\FireRun.py", line 630, in constrainByShape_Run
    preprocess.preprocess_region_t(t, region=region, read_r

In [9]:
logger.file_handler

AttributeError: 'Logger' object has no attribute 'file_handler'

Next, I want to make any logs that are specific to a fire go into the FEDSoutput subdirectories. I'll need to modify the logging configurations in `FireMain` and `postprocess`. I created a constant (`LOG_SUBDIR`) in `FireConsts` to indicate if subdirectories are ever used. While the default is `False` I changed it to `True` in my `.env` file. Also, delete any postprocessed and final outputs from this fire to start from a clean slate. 